## Building Neural Network Model for Learning Heston Pricing Simulation method

---


|  |  |
|:---|:---|
|**Reading Time** |  10 minutes |
|**Packages used** | Defined with relevant DocStrings within repository   |
|**Outcome** | Trained NN model save in the directory ./model/output | 

---

### First step is to import all relevant packages to train neural network model. 

### Note that in the repository under the directory ./model/implied_volatility/core/, I have created a module heston_model_pricing which contains get_heston_price method that generates simultaneous simulation of stock price and volatility process to compute option price under heston model using numerical method. 

---

### Although majority of option parameters require to compute heston price are self-explanatory, some of them that are explicitely required for evolution of stochastic process and compute price using numerical method are as follows:- 

|  |  |
|:---|:---|
|**Initial variance** |  Starting variance at the start of simulation period |
|**Long average variance** | Long term average variance of underlying volatility |
|**Reversion speed** | Volatility evolution under Stochastic process follows mean-reverting process, speed signifies how strong it is|
|**Correlation** | Correlation among underlying stochastic price and volatility evolution weiner process, used to get random value from the joint distribution|
|**Volatility of the variance** | Used for simulation volatility evolution process |



---

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import json

import warnings
warnings.filterwarnings("ignore") # just to remove clutter in the notebook

In [2]:
os.chdir("..")

In [3]:
from model.pricing.utils.get_data import get_current_price, pre_processing
from model.pricing.core.neural_network import run_nn_model
from model.implied_volatility.core.heston_model_pricing import create_heston_dataset

### Second step does the following:- 

- Define data size, read ticker information, and retrieve current price for that ticker. As an example, I have used S&P500
- Create a diversified set of input parameters using Latin Hypercube sampling to cover range of possible input parameters. This model contains wide range of input parameters as compare to building BS pricing nn model as it also contains parameters as explined above
- Compute heston price using numerical method and then compute implied volatility using brent method which is derivative free as Newton Raphson method generally blows up as vega is close to 0 for deep OTM and ITM options. 
- Read already tuned hyperparmeters. Method used to identify these hyper parameters is Random search using cross-validation, same hyper parameters are used as per BS neural network model

---

In [ ]:
data_set_size = 10000
df, st_current_price, range_of_inputs = pre_processing(data_set_size, "HESTON")
dt_set = create_heston_dataset(df, st_current_price, range_of_inputs)

In [ ]:
tuned_hyper_parameter_file_name = os.getcwd() + "/model/output/" + "best_hyper_parameter.json"
with open(tuned_hyper_parameter_file_name) as f:
    df_hyper = json.load(f)

### Third step does the following:- 

- Define columns used as input features and the target variable, much more input features as compare to BS nn model.
- Run the model using the cycilical decaying learning rate, more detail is avaiable in the DocString of run_nn_model method
- Plot the loss on the training set and validation with each epoch
- The below plot demonstrates that model is not overfitting as loss is decreasing in both training and validation set

In [ ]:
feature_columns = ['moneyness', 'time_to_maturity', 'risk_free_rate', 'correlation', 'reversion_speed',
                       'Long_average_variance', 'vol_vol', 'initial_variance']
target = 'opt_price_by_strike'
model, training_history = run_nn_model(dt_set, df_hyper, feature_columns, target)

In [ ]:
train_log_mse = [np.log(i) for i in training_history.history['mse']]
validation_log_mse = [np.log(i) for i in training_history.history['val_mse']]
plt.plot(train_log_mse)
plt.plot(validation_log_mse)
plt.title('Model Loss on training set and validation set')
plt.ylabel('log(MSE)')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

#### Last step is to save the model in the ./model/output directory

In [ ]:
model_save_path = os.getcwd() + "/model/output/" + "Heston_NN_model.h5"
model.save(model_save_path)